In [1]:
from math import log
import re
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
from flask import Flask, request, render_template
app = Flask(__name__)
import collections
from collections import defaultdict, Counter

In [2]:
def lemmatize(text): # лемматизатор, на вход принимает запрос, очищенный от стоп-слов
    t = []
    for i in text.split():
        i = i.strip(',\.\\/\]\[ *!?-_()#$%²³^&:;«»"\n')
        word = morph.parse(i)[0]
        t.append(word.normal_form)
    return t
        

In [3]:
k1 = 2.0
b = 0.75

def score_BM25(n, qf, N, dl, avdl): 
    
    """
    n - количество доков, содержащих слово
    qf - частота слова
    N - общее количество доков в коллекции
    dl - длина дока (количество слов в доке)
    avdl - средняя длина дока
    """

    K = compute_K(dl, avdl)
    IDF = log((N - n + 0.5) / (n + 0.5))
    frac = ((k1 + 1) * qf) / (K + qf)
    return IDF * frac

def compute_K(dl, avdl):
    return k1 * ((1-b) + b * (float(dl)/float(avdl)))

In [4]:
def inverted_ind(doc): #создаёт инвертированный индекс, на вход принимает список с текстами
    d = {}
    for n in range(0, len(doc)):
        for term in doc[n]:
            if term not in d:
                d[term] = [n+1]
            else:
                d[term].append(n+1)
    return d


In [5]:
def freq(word, ind): #вычисляет частоты
    freq = {}
    if word in ind:
        for i in ind[word]:
            #print(i)
            if i in freq:
                freq[i] += 1
            else:
                freq[i] = 1

    return freq 


In [6]:
def search_q(query):
    stop_words = ['большой', 'бы', 'быть', 'в', 'весь', 'вот', 'все', 'всей', 'вы', 'говорить', 'год', 'да', 'для', 'до', 'еще', 'ещё', 'же', 'знать', 'и', 'из', 'к', 'как', 'который', 'мочь', 'мы', 'на', 'наш', 'не', 'него', 'нее', 'неё', 'нет', 'них', 'но', 'о', 'один', 'она', 'они', 'оно', 'оный', 'от', 'ото', 'по', 'с', 'свой', 'себя', 'сказать', 'та', 'такой', 'только', 'тот', 'ты', 'у', 'что', 'это', 'этот', 'я']
    query_cleaned = ' '.join([word for word in query.split() if word not in stop_words])
    q = lemmatize(query_cleaned)
    collection = []
    dls = {}                        # dls - словарь {документ : длина документа (количество слов в нём)}
    for doc in range(1, 1000):
        file_1 = open(str(doc) + '.txt', 'r', encoding='utf-8')
        text = file_1.read()
        lemmas = lemmatize(text)
        collection.append(lemmas)
        dls[doc] = len(lemmas)
        file_1.close()
        
    avdl = sum(dls)/len(dls)        # avdl - средняя длина дока
    qf = {}
    d = {}
    ind = inverted_ind(collection)
    for word in q:
        if word in ind:
            n = len(set(ind[word]))
            qf = freq(word, ind)
            for i in qf:
                for j in dls:
                    if i == j:
                        d[j] = score_BM25(n, i, len(collection), j, avdl)
    
    sc = []
    for i in d:
        sc.append(d[i])
    sc.sort(reverse=True)
    t = sc[:10]
    links = []
    for i in t:
        for j in d:
            if i == d[j]:
                file = open(str(j) + '.txt', 'r', encoding='utf-8')
                txt = file.read()
                l = re.findall('@url (https://.+)\n', txt)
                links.append((l[0], str(i)))
                
    return links

In [ ]:
@app.route('/', methods=['get','post'])
def index():
    if request.form:
        query = request.form['query']
        links = search_q(query)
        return render_template('index.html',**locals())
    return render_template('index.html')

if __name__ == '__main__':
    app.run()

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Oct/2017 13:47:52] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2017 13:47:59] "POST / HTTP/1.1" 200 -
